<a href="https://colab.research.google.com/github/Misudhari/Deep-Learning-Lab/blob/main/1132210046_Mithilesh_Chaudhari_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mithilesh Chaudhari 
# 1132210046
# Roll no : 11

In [ ]:
!pip install keras_tuner

### Importing libraries 

In [ ]:
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from keras_tuner.tuners import RandomSearch
from keras.layers import Dense,Dropout,Flatten,ReLU,LeakyReLU
from keras import layers

### Loading data  

In [ ]:
data = pd.read_csv("data.csv")

In [ ]:
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,313000.0,3,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2384000.0,5,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,342000.0,3,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,420000.0,3,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,550000.0,4,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [ ]:
data.isna().sum()

price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
street           0
city             0
statezip         0
country          0
dtype: int64

### Dropping Unnecessary columns

In [ ]:
drop_cols = ["country","street","city","statezip"]
data.drop(columns = drop_cols,inplace = True)

In [ ]:
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
0,313000.0,3,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005
1,2384000.0,5,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0
2,342000.0,3,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0
3,420000.0,3,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0
4,550000.0,4,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992


In [ ]:
data["yr_old"] = 2022-data["yr_built"]
data["Renovate"] = np.where(data["yr_renovated"]>0,1,0)

In [ ]:
data.drop(columns = ["yr_built","yr_renovated"],inplace = True)

In [ ]:
data.corr()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_old,Renovate
price,1.000000,0.200336,0.327110,0.430410,0.050451,0.151461,0.135648,0.228504,0.034915,0.367570,0.210427,-0.021857,-0.028595
bedrooms,0.200336,1.000000,0.545920,0.594884,0.068819,0.177895,-0.003483,0.111028,0.025080,0.484705,0.334165,-0.142461,-0.060720
bathrooms,0.327110,0.545920,1.000000,0.761154,0.107837,0.486428,0.076232,0.211960,-0.119994,0.689918,0.298020,-0.463498,-0.214931
sqft_living,0.430410,0.594884,0.761154,1.000000,0.210538,0.344850,0.117616,0.311009,-0.062826,0.876443,0.447206,-0.287775,-0.122265
sqft_lot,0.050451,0.068819,0.107837,0.210538,1.000000,0.003750,0.017241,0.073907,0.000558,0.216455,0.034842,-0.050706,-0.023187
floors,0.151461,0.177895,0.486428,0.344850,0.003750,1.000000,0.022024,0.031211,-0.275013,0.522814,-0.255510,-0.467481,-0.232399
waterfront,0.135648,-0.003483,0.076232,0.117616,0.017241,0.022024,1.000000,0.360935,0.000352,0.078911,0.097501,0.023563,0.008503
view,0.228504,0.111028,0.211960,0.311009,0.073907,0.031211,0.360935,1.000000,0.063077,0.174327,0.321602,0.064465,0.022860
condition,0.034915,0.025080,-0.119994,-0.062826,0.000558,-0.275013,0.000352,0.063077,1.000000,-0.178196,0.200632,0.399698,-0.185369
sqft_above,0.367570,0.484705,0.689918,0.876443,0.216455,0.522814,0.078911,0.174327,-0.178196,1.000000,-0.038723,-0.408535,-0.159518


### Spliting data 

In [ ]:
x = data.drop(columns = ["price"])
y = data[["price"]]

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
ms = MinMaxScaler()
sc = StandardScaler()
x = ms.fit_transform(np.array(x))
y = ms.fit_transform(np.array(y))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)

In [ ]:
import tensorflow as tf

### Building model

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers',2, 20)):
        model.add(layers.Dense(units = hp.Int('Units_' + str(i), min_value = 32, max_value = 512, step = 30), 
                               activation='relu'))
        model.add(layers.Dense(units = 1, activation = 'linear'))
        model.compile(
            optimizer = tf.keras.optimizers.Adam(
                hp.Choice('Learning_rate', [1e-2, 1e-3, 1e-4])), 
            loss = 'mse',
            metrics = ['mse'])
    return model

In [ ]:
tunner = RandomSearch(build_model, 
                      objective = 'val_mse',
                      max_trials = 5,
                      executions_per_trial = 3
                     )

In [ ]:
tunner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 5 Complete [00h 00m 33s]
val_mse: 0.00017296829416106144

Best val_mse So Far: 0.00010749481831832479
Total elapsed time: 00h 02m 15s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tunner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 2
Units_0: 92
Learning_rate: 0.01
Units_1: 62
Score: 0.00010749481831832479
Trial summary
Hyperparameters:
num_layers: 6
Units_0: 482
Learning_rate: 0.0001
Units_1: 122
Units_2: 302
Units_3: 62
Units_4: 422
Units_5: 362
Units_6: 182
Units_7: 362
Units_8: 362
Units_9: 422
Units_10: 272
Units_11: 482
Units_12: 272
Units_13: 332
Units_14: 302
Units_15: 272
Units_16: 182
Units_17: 122
Score: 0.00012282053648959845
Trial summary
Hyperparameters:
num_layers: 19
Units_0: 362
Learning_rate: 0.01
Units_1: 512
Units_2: 212
Units_3: 152
Units_4: 122
Units_5: 422
Units_6: 62
Units_7: 482
Units_8: 212
Units_9: 182
Units_10: 302
Units_11: 152
Units_12: 332
Units_13: 392
Units_14: 152
Units_15: 512
Units_16: 212
Units_17: 332
Units_18: 32
Score: 0.00017282820772379637
Trial summary
Hyperparameters:
num_layers: 18
Units_0: 62
Learning_rate: 0.0001
Units_1: 362
Units_2: 32
Units_3: 32
Units_4

In [ ]:
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(212,input_dim = x_train.shape[1],activation = "relu"))
model.add(Dense(332,activation = "relu"))
model.add(Dense(362,activation = "relu"))
model.add(Dense(212,activation = "relu"))
model.add(Dense(362,activation = "relu"))
model.add(Dense(452,activation = "relu"))
model.add(Dense(152,activation = "relu"))
model.add(Dense(182,activation = "relu"))
model.add(Dense(272,activation = "relu"))
model.add(Dense(32,activation = "relu"))
model.add(Dense(1,activation = "linear"))

In [ ]:
model.compile(loss = "mse",optimizer = tf.keras.optimizers.Adam(lr = 0.001),metrics = ["mse"])

C:\Users\SHREYA\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(x_train, y_train, epochs = 50,batch_size = 20,validation_data =  (x_test,y_test))

Epoch 1/50
184/184 [==============================] - 4s 13ms/step - loss: 4.8968e-04 - mse: 4.8968e-04 - val_loss: 1.0805e-04 - val_mse: 1.0805e-04
Epoch 2/50
184/184 [==============================] - 2s 11ms/step - loss: 4.7418e-04 - mse: 4.7418e-04 - val_loss: 1.7222e-04 - val_mse: 1.7222e-04
Epoch 3/50
184/184 [==============================] - 2s 12ms/step - loss: 4.8352e-04 - mse: 4.8352e-04 - val_loss: 1.1546e-04 - val_mse: 1.1546e-04
Epoch 4/50
184/184 [==============================] - 2s 12ms/step - loss: 4.7026e-04 - mse: 4.7026e-04 - val_loss: 1.0919e-04 - val_mse: 1.0919e-04
Epoch 5/50
184/184 [==============================] - 2s 11ms/step - loss: 4.4675e-04 - mse: 4.4675e-04 - val_loss: 9.6839e-05 - val_mse: 9.6839e-05
Epoch 6/50
184/184 [==============================] - 2s 12ms/step - loss: 4.5847e-04 - mse: 4.5847e-04 - val_loss: 1.4207e-04 - val_mse: 1.4207e-04
Epoch 7/50
184/184 [==============================] - 2s 12ms/step - loss: 4.5667e-04 - mse: 4.5667e-04 - 

In [ ]:
pred = model.predict(x_test)

29/29 [==============================] - 1s 6ms/step


In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,pred)

-0.27595325978149243